In [ ]:
import GSL: hypergeom, sf_gamma

mybesselj(α, x) = (x / 2)^α / sf_gamma(α + 1) * hypergeom([], α + 1, -x^2 / 4)
function sphericalbesselj_1(n, x::AbstractFloat)
    if iszero(x)
        return iszero(n) ? one(x) : zero(x)
    else
        return √(π / 2x) * mybesselj(n + 0.5, x)
    end
end


In [ ]:
import SpecialFunctions
sphericalbesselj_2(ν, x) = √(π/2x)*SpecialFunctions.besselj(ν+1/2, x)

In [ ]:
using PyCall
scipy_special = pyimport("scipy.special")

In [ ]:
n = 100
x = 10000.0
# Mathematica: -0.000072814706186135590960089481513765273217282769121987832010967...
println(sphericalbesselj_1(n, x))
println(sphericalbesselj_2(n, x))
println(scipy_special.spherical_jn(n, x))

In [ ]:
n = 10
x = 10000.0
# Mathematica: 0.0000310846680541186048422278797019184103613015346166689343934711
println(sphericalbesselj_1(n, x))
println(sphericalbesselj_2(n, x))
println(scipy_special.spherical_jn(n, x))